In [1]:
import os
import csv, sys
# ^ necessary for command-line args.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import PMM
MAJOR_PARTY_THRESH = 0.05

In [53]:
# ================ import data ===============
pathstr="data/raw_2019_copy/"
f_qlist_out     = os.path.join( pathstr, "PMM_qlist.tsv")
f_standings_out = os.path.join( pathstr, "PMM_standings.tsv")

T3_path = os.path.join(pathstr, "table_tableau03.csv")
T7_path = os.path.join(pathstr, "table_tableau07.csv")        
T8_path = os.path.join(pathstr, "table_tableau08.csv")        

# info on total # votes cast & turnout.
df_Nvotes     = pd.read_csv(T3_path, index_col="Province")

# info on seats awarded
Seats_init = pd.read_csv(T7_path, index_col="Province")

# Valid votes by party:
VV_bp      = pd.read_csv( T8_path, index_col=0)

In [3]:
Const_Seats = PMM.get_party_seat_standings(Seats_init)
Seats_total_init = sum( Const_Seats )
# Const_Seats

In [4]:
# get the total num votes valid, invalid
N_total_votes = df_Nvotes.iloc[:, 6].sum()
print("Total votes : %d " %(N_total_votes) )
print( "Percent votes valid  : %.4f " %( 100*df_Nvotes.iloc[:, 2].sum()/N_total_votes) )
print( "Percent votes invalid: %.4f " %( 100*df_Nvotes.iloc[:, 4].sum()/N_total_votes) )

Total votes : 18350359 
Percent votes valid  : 99.0219 
Percent votes invalid: 0.9781 


In [5]:
Pop_vote_share = pd.Series( VV_bp.sum(axis=1)/N_total_votes,
                            index= VV_bp.index )
# Pop_vote_share

In [6]:
# Select parties over threshold support
maj_parties = Pop_vote_share.index[ Pop_vote_share >= MAJOR_PARTY_THRESH ]
Vote_counts = VV_bp.loc[maj_parties,].sum(axis=1)

Vote_counts.index = [ PMM.party_abbrev[key] for key in Vote_counts.index ]
# All other parties are grouped together as "OTHER"
# i.e. those who are _explicitly_ independent, as well as those whose 
# parties are relegated to "independence" by falling below threshold support.
Vote_counts["OTH"] = sum ( VV_bp.loc[Pop_vote_share < MAJOR_PARTY_THRESH, ].sum(axis = 1 ) )

# "SPL" captures all rejected (i.e. 'spoiled') ballots 
Vote_counts["SPL"] = df_Nvotes.sum(axis=0)[4]

Vote_counts

BLQ    1387030.0
CON    6239227.0
GRN    1189607.0
LIB    6018728.0
NDP    2903722.0
OTH     432566.0
SPL     179479.0
dtype: float64

In [7]:
Standings = pd.DataFrame( {"Votes":Vote_counts, "Seats_init": Const_Seats } )
Standings

,Votes,Seats_init
BLQ,1387030.0,32
CON,6239227.0,121
GRN,1189607.0,3
LIB,6018728.0,157
NDP,2903722.0,24
OTH,432566.0,1
SPL,179479.0,0


In [8]:
all_parties = [PMM.party( Standings.index[p], 
                          Standings.iloc[p,0], 
                          Standings.iloc[p,1],
                          N_total_votes,
                          Seats_total_init) 
               for p in range(Standings.shape[0])]
namelist    = Standings.index
Num_parties = len(all_parties)

In [20]:
Total_quotient_list=[]
for pind in range(Num_parties):
    Total_quotient_list.extend( all_parties[pind].party_quotient_list )
Total_quotient_list.sort(reverse=True)

In [61]:
shortlist = Total_quotient_list[:2*Seats_total_init]
len( shortlist) 
Qlist = pd.DataFrame({"j" :[ q.jval   for q in shortlist],
              "Value"     :[ q.value  for q in shortlist],
              "Assigned"  :[ int(q.assigned) for q in shortlist],
              "party"     :[ q.party_att     for q in shortlist],
              } )

In [62]:
Qlist.round(2).to_csv(f_qlist_out, sep="\t")
# f_qlist_out     

In [58]:
np.pi.round(3)

AttributeError: 'float' object has no attribute 'round'

In [63]:
Seats_total_init

338

In [64]:
Threshold = Total_quotient_list[Seats_total_init-1].value
total_seats_assigned = Seats_total_init

# Starting from the first unassigned seat
sval = Seats_total_init
